In [ ]:
#@title Imports
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.colheader_justify', 'center')
import numpy as np
import random
import time
import re

import scipy.stats
from scipy import stats
from sklearn import linear_model
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2

from inspect import signature

def someMethod(self, arg1, kwarg1=None):
    pass

possession_data_seasons = np.arange(1974, 2023, 1)
play_by_play_data_seasons = np.arange(1997, 2023, 1)
pre_possesson_data_seasons = np.arange(1952, 1974, 1)
aba_seasons = np.arange(1974, 1977, 1)
aba_pre_possession_seasons = np.arange(1968, 1974, 1)

# plot
import seaborn as sns
import ast
from matplotlib.lines import Line2D
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
sns.set_style('darkgrid')        # darkgrid, white grid, dark, white and ticks
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.rc('font', size=13)          # controls default text sizes

def label_point(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
      #if i % 2 == 0:  
      ax.text(point['x']+0.12, point['y']+.25, str(point['val']))
      #else:
      #ax.text(point['x']+0.30, point['y'], str(point['val']))
      #ax.text(point['x']+0.05, point['y'], str(point['val']))

def label_point_year(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
      #if i % 2 == 0: 
      ax.text(point['x']+0.12, point['y'], int(point['val']))
      #else:
      #ax.text(point['x']+0.30, point['y'], str(point['val']))
      #ax.text(point['x']+0.05, point['y'], str(point['val']))

In [ ]:
#@title Import Selenium
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [ ]:
#@title Start Webdriver and VirtualDisplay
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

url = "http://example.com" 
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")

options.headless = True

wd = webdriver.Chrome("/usr/bin/chromedriver", options=options)

!pip install pyvirtualdisplay
!apt-get install xvfb
from pyvirtualdisplay import Display
display = Display(visible=0, size=(800, 800))  
display.start()



In [ ]:
#@title WOWY Two Guys On
def scrape_nba_possession_97_00_wowy_data(url, player1, player_ref1, player2, player_ref2, finals_series):
                       
  wd.get(url)   
  html = wd.page_source
  soup = BeautifulSoup(html, features="lxml")
  time.sleep(10)

  for first_div in soup.find_all('div', attrs={'id': 'content'}):
          second_div = first_div.find('div', attrs={'id': 'div_other_scores'})
          hrefs = second_div.findAll('td', attrs={'class': 'right gamelink'})
          hrefs = str(hrefs)
          ref_urls = re.findall(r'/\d+\w+\D\w+', hrefs)
          urls = []
          for ref_url in ref_urls:
            ref_url = "https://www.basketball-reference.com/boxscores/pbp" + ref_url
            urls.append(ref_url)

  stint_list_df = pd.DataFrame(columns = ['Game', 'Visiting Team +/-', 'Possessions'])
  stint_list_df = stint_list_df.iloc[0:0]
  series_possessions = 0
  arena1_margin = 0
  arena2_margin = 0
  game_count = 1
  for game_url in urls:

    wd.get(game_url)   
    html = wd.page_source
    soup = BeautifulSoup(html, features="lxml")

    for first_div in soup.find_all('div', attrs={'id': 'content'}):
            second_div = first_div.find('div', attrs={'id': 'all_pbp'})
            first_table = second_div.find('table', attrs={'id': 'pbp'})
            body = first_table.find('tbody')
    # grab rows
    rows = body.findAll('tr')[0:]

    #here#

    # Default possessions
    game_possessions = 0
    pbp_poss = 0
    stint_poss = 0
    secret_stint_poss = 0
    player_secretly_in_game_extra_poss = 0

    player1_out_wait = 0
    player2_out_wait = 0
    
    margin = 0
    game_margin = 0
    initial_margin = 0

    grab_backup_score = 0
    grab_next_half_score = 0

    p1_quarter_check = 0
    p2_quarter_check = 0

    p1_enter_q = 0
    p2_enter_q = 0

    quarter_conundrum = 0
    use_q = 0

    possession_enders = ['Defensive rebound', 'makes 2-pt', 'makes 3-pt', 'Turnover', 'Shooting foul']
    fts_no_shooting_foul = ['free throw 1 of 2']

    player1_out = [f'enters the game for <a href=\"{player_ref1}\">{player1}']
    player1_in = [f'{player1}']
    player1_enter = [f'{player1}</a> enters the game']

    player2_out = [f'enters the game for <a href=\"{player_ref2}\">{player2}']
    player2_in = [f'{player2}']
    player2_enter = [f'{player2}</a> enters the game']

    quarter_start = ['Start of 2nd quarter', 'Start of 4th quarter']
    half_start = ['Start of 3rd quarter']

    end_of_q = ['End of 1st quarter', 'End of 2nd quarter', 'End of 3rd quarter']

    dont_double_end_of_q = 0
    dont_do_4q = 0

    finish_with_q = 0


    for row in rows:
      row = str(row)

      if 'Start of 4th quarter' in row:
        dont_do_4q = 1

      if any(iny in row for iny in player1_enter):
        p1_enter_q = 1
        if p2_enter_q == 1:
          backup_score = re.search("\d+-\d+", str(row))
          backup_score = backup_score[0]

          score1 = backup_score.split('-')[0]
          score2 = backup_score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          backup_margin = score1 - score2
          initial_margin = backup_margin
      if any(iny in row for iny in player2_enter):
        p2_enter_q = 1
        if p1_enter_q == 1:
          backup_score = re.search("\d+-\d+", str(row))
          backup_score = backup_score[0]

          score1 = backup_score.split('-')[0]
          score2 = backup_score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          backup_margin = score1 - score2
          initial_margin = backup_margin

      # Q ending
      if '0:00.0' in row and not any(iny in row for iny in end_of_q) and dont_double_end_of_q == 0 and dont_do_4q == 0 and '10:00.0' not in row:
        if player2_out_wait == 0 and player1_out_wait == 0:
          score = re.search("\d+-\d+", str(row))
          score = score[0]
          print(f"Q ended with {player1} on and {player2} on {score}")
          score1 = score.split('-')[0]
          score2 = score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          margin = score1 - score2
          margin = margin - initial_margin
          print(f"\n{margin} +/- stint.\n")
          stint_list_df.loc[len(stint_list_df)] = [game_count, margin, int(stint_poss/2)]
          stint_poss = 0
          game_margin = game_margin + margin
          dont_double_end_of_q = 1

          p1_enter_q = 0
          p1_enter_q = 0

          p1_quarter_check = 0
          p2_quarter_check = 0


          backup_score = re.search("\d+-\d+", str(row))
          backup_score = backup_score[0]

          score1 = backup_score.split('-')[0]
          score2 = backup_score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          backup_margin = score1 - score2
          initial_margin = backup_margin
          q_margin = backup_margin
          q_score = backup_score

      # Q ending
      elif any(iny in row for iny in end_of_q) and dont_double_end_of_q == 0 and dont_do_4q == 0:
        if player2_out_wait == 0 and player1_out_wait == 0:
          score = re.search("\d+-\d+", str(prev_row))
          score = score[0]
          print(f"Q ended with {player1} on and {player2} on {score}")
          score1 = score.split('-')[0]
          score2 = score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          margin = score1 - score2
          margin = margin - initial_margin
          print(f"\n{margin} +/- stint.\n")
          stint_list_df.loc[len(stint_list_df)] = [game_count, margin, int(stint_poss/2)]
          stint_poss = 0
          game_margin = game_margin + margin
          dont_double_end_of_q = 1

          p1_enter_q = 0
          p1_enter_q = 0

          p1_quarter_check = 0
          p2_quarter_check = 0

          backup_score = re.search("\d+-\d+", str(prev_row))
          backup_score = backup_score[0]

          score1 = backup_score.split('-')[0]
          score2 = backup_score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          backup_margin = score1 - score2
          initial_margin = backup_margin
          q_margin = backup_margin
          q_score = backup_score
        else:

          p1_enter_q = 0
          p1_enter_q = 0

          p1_quarter_check = 0
          p2_quarter_check = 0

          backup_score = re.search("\d+-\d+", str(prev_row))
          backup_score = backup_score[0]

          score1 = backup_score.split('-')[0]
          score2 = backup_score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          backup_margin = score1 - score2
          initial_margin = backup_margin
          q_margin = backup_margin
          q_score = backup_score



      if any(iny in row for iny in quarter_start) or any(iny in row for iny in half_start):
        dont_double_end_of_q = 0

      # 1/2 2nd half is starting; presume both players are now on the floor.
      if any(iny in row for iny in half_start):
        grab_next_half_score = 1
        if player2_out_wait == 1 or player1_out_wait == 1:
          player1_out_wait = 0
          player2_out_wait = 0
        continue
      # 2/2 2nd half is starting; presume both players are now on the floor.
      if grab_next_half_score == 1:
        score = re.search("\d+-\d+", str(row))
        score = score[0]
        print(f"Half started, {player1} is on and {player2} is on. {score}")

        score1 = score.split('-')[0]
        score2 = score.split('-')[1]
        score1 = int(score1)
        score2 = int(score2)

        initial_margin = score1 - score2
        player_secretly_in_game_extra_poss = 0
        secret_stint_poss = 0
        grab_backup_score = 0

        grab_next_half_score = 0

      # 1/2 a quarter is starting; grab score in case a player has entered/left game without us knowing
      if any(iny in row for iny in quarter_start):
        grab_backup_score = 1
        p1_quarter_check = 0
        p2_quarter_check = 0
        continue

      # 2/2 grab score from next row as 'Q starting row' displays no score
      if grab_backup_score == 1:
        
        player_secretly_in_game_extra_poss = 0
        secret_stint_poss = 0
        grab_backup_score = 0
        player1_out_wait = 1
        player2_out_wait = 1
        p1_never_left_game = 1
        p2_never_left_game = 1

        quarter_conundrum = 1

      # player 1 is in the game (not entering) and was believed to be out of the game
      if any(iny in row for iny in player1_in) and not any(outy in row for outy in player1_out) and player1_out_wait == 1 and not any(iny in row for iny in player1_enter):
          player1_out_wait = 0
          p1_quarter_check = 1

          # player 1 and 2 were both on at start of Q
          if player2_out_wait == 0 and p2_quarter_check == 1 and p2_enter_q == 0:
            print(f"{player1} on and {player2} on. {q_score}")
            finish_with_q = 1
          # player 1 is in game after start of a quarter player 2 is out of the game
          # we can at least confirm both players' states
          elif quarter_conundrum == 1 and p2_quarter_check == 1 and p2_quarter_check == 1:
            p1_quarter_check = 0
            p2_quarter_check = 0
            quarter_conundrum = 0
          # player 1 was on at the start of the Q and player 2 entered after the Q began
          elif p2_enter_q == 1 and player2_out_wait == 0:
            print(f"{player1} on and {player2} on. {backup_score}")
            use_q = 1


      # player 2 is in the game (not entering) and was believed to be out of the game
      if any(iny in row for iny in player2_in) and not any(outy in row for outy in player2_out) and player2_out_wait == 1 and not any(iny in row for iny in player2_enter):
          player2_out_wait = 0
          p2_quarter_check = 1

          # player 1 and 2 were both on at start of Q
          if player1_out_wait == 0 and p1_quarter_check == 1 and p1_enter_q == 0:
            print(f"{player1} on and {player2} on. {q_score}")
            finish_with_q = 1
          
          # player 2 is in game after start of a quarter player 1 is out of the game
          # we can at least confirm both players' states
          elif quarter_conundrum == 1 and p1_quarter_check == 1 and p1_quarter_check == 1:
            p1_quarter_check = 0
            p2_quarter_check = 0
            quarter_conundrum = 0
          # player 2 was on at the start of the Q and player 1 entered after the Q began
          elif p1_enter_q == 1 and player1_out_wait == 0:
            print(f"{player1} on and {player2} on. {backup_score}")
            use_q = 1


      # player 2 is exiting the game
      if any(outy in row for outy in player2_out):
        player2_out_wait = 1
        # need to use score from the beginning of the quarter as we didn't realize both players were in at the start
        if use_q == 1 and player1_out_wait == 0 and not any(iny in row for iny in player1_enter):
          use_q = 0
          if p1_enter_q == 1 or p2_enter_q == 1:
            score = re.search("\d+-\d+", str(row))
            score = score[0]
            print(f"{player1} on and {player2} off. {score}")
            score1 = score.split('-')[0]
            score2 = score.split('-')[1]
            score1 = int(score1)
            score2 = int(score2)

            margin = score1 - score2
            margin = margin - initial_margin
            print(f"\n{margin} +/- stint.\n")
            stint_list_df.loc[len(stint_list_df)] = [game_count, margin, int(stint_poss/2)]
            stint_poss = 0
            game_margin = game_margin + margin
          else:
            print(f"{player1} on and {player2} off. {q_score}")
            print(f"\n{q_margin} +/- stint.\n")
            stint_list_df.loc[len(stint_list_df)] = [game_count, q_margin, int(stint_poss/2)]
            stint_poss = 0
            game_margin = game_margin + q_margin

        # player 1 is in the game and is not checking out
        elif player1_out_wait == 0 and not any(outy in row for outy in player1_out):
          score = re.search("\d+-\d+", str(row))
          score = score[0]
          print(f"{player1} on and {player2} off. {score}")
          score1 = score.split('-')[0]
          score2 = score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          margin = score1 - score2
          margin = margin - initial_margin
          print(f"\n{margin} +/- stint.\n")
          stint_list_df.loc[len(stint_list_df)] = [game_count, margin, int(stint_poss/2)]
          stint_poss = 0
          game_margin = game_margin + margin


      # player 1 is listed on this row and is not checking out
      if any(iny in row for iny in player1_in) and not any(outy in row for outy in player1_out):
        # player 1 is checking in and player 2 is also on
        # therefore grab score as a stint with player 1 on and player 2 off is starting
        if player1_out_wait == 1 and player2_out_wait == 0 and not any(iny in row for iny in player2_enter):
          score = re.search("\d+-\d+", str(row))
          score = score[0]
          print(f"{player1} on and {player2} on. {score}")

          score1 = score.split('-')[0]
          score2 = score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          margin = score1 - score2
          initial_margin = margin
        player1_out_wait = 0

      # player 2 is listed on this row and is not checking out
      if any(iny in row for iny in player2_in) and not any(outy in row for outy in player2_out):
      # player 2 is checking in and player 1 was on the floor and is not checking out
      # therefore update margin as a stint with player 1 on and player 2 off is ending.
        if player2_out_wait == 1 and player1_out_wait == 0 and not any(iny in row for iny in player1_enter) and not any(outy in row for outy in player1_out):
          score = re.search("\d+-\d+", str(row))
          score = score[0]
          print(f"{player1} on and {player2} on. {score}")
          score1 = score.split('-')[0]
          score2 = score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          margin = score1 - score2
          initial_margin = margin
        player2_out_wait = 0

      # player 1 is exiting game
      if any(outy in row for outy in player1_out):
        player1_out_wait = 1
        # need to use score from the beginning of the quarter as we didn't realize both players were in at the start
        if use_q == 1  and player2_out_wait == 0 and not any(iny in row for iny in player2_enter):
          use_q = 0
          print(f"{player1} off and {player2} on. {q_score}")
          print(f"\n{q_margin} +/- stint.\n")
          stint_list_df.loc[len(stint_list_df)] = [game_count, q_margin, int(stint_poss/2)]
          stint_poss = 0
          game_margin = game_margin + q_margin
        # player 2 is in game
        elif player2_out_wait == 0 and not any(iny in row for iny in player2_enter):
          score = re.search("\d+-\d+", str(row))
          score = score[0]
          print(f"{player1} off and {player2} on. {score}")
          score1 = score.split('-')[0]
          score2 = score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          margin = score1 - score2
          margin = margin - initial_margin
          print(f"\n{margin} +/- stint.\n")
          stint_list_df.loc[len(stint_list_df)] = [game_count, margin, int(stint_poss/2)]
          stint_poss = 0
          game_margin = game_margin + margin


      if any(end_poss in row for end_poss in possession_enders) and player1_out_wait == 0 and player2_out_wait == 0:
        pbp_poss = pbp_poss + 1
        stint_poss = stint_poss + 1
      elif any(end_poss in row for end_poss in possession_enders) and (player1_out_wait == 0 or player2_out_wait == 0):
        player_secretly_in_game_extra_poss = player_secretly_in_game_extra_poss + 1
        secret_stint_poss = secret_stint_poss + 1

      prev_row = row



    if player2_out_wait == 0 and player1_out_wait == 0:

      last_row = body.findAll('tr')[-1]
      score = re.search("\d+-\d+", str(last_row))
      score = score[0]

      score1 = score.split('-')[0]
      score2 = score.split('-')[1]
      score1 = int(score1)
      score2 = int(score2)

      margin = score1 - score2

      if use_q == 1 and p1_enter_q == 0 and p2_enter_q == 0:
          use_q = 0
          margin = margin - q_margin
          print(f"{player1} on and {player2} on. {q_score}")
          last_row = body.findAll('tr')[-1]
          score = re.search("\d+-\d+", str(last_row))
          score = score[0]
          print(f"Final Score: {score}")
          print(f"\n{margin} +/- stint.\n")
          stint_list_df.loc[len(stint_list_df)] = [game_count, margin, int(stint_poss/2)]
          stint_poss = 0
          game_margin = game_margin + margin


      elif finish_with_q == 1:
          use_q = 0
          margin = margin - q_margin
          last_row = body.findAll('tr')[-1]
          score = re.search("\d+-\d+", str(last_row))
          score = score[0]
          print(f"Final Score: {score}")
          print(f"\n{margin} +/- stint.\n")
          stint_list_df.loc[len(stint_list_df)] = [game_count, margin, int(stint_poss/2)]
          stint_poss = 0
          game_margin = game_margin + margin
      

      else:
        print(f"Final Score: {score}")
        margin = margin - initial_margin
        print(f"\n{margin} +/- stint.\n")
        stint_list_df.loc[len(stint_list_df)] = [game_count, margin, int(stint_poss/2)]
        stint_poss = 0
        game_margin = game_margin + margin
    
    if finals_series == 0:

      if game_count == 1 or game_count == 2 or game_count == 5 or game_count == 7:
        arena1_margin = arena1_margin + game_margin
      if game_count == 3 or game_count == 4 or game_count == 6:
        arena2_margin = arena2_margin + game_margin

      game_possessions = game_possessions + pbp_poss
      series_possessions = series_possessions + game_possessions
      if game_margin >= 0:
        print(f"\nThis game the road team was +{game_margin} with {player1} on and {player2} on.\n")
      else:
        print(f"\nThis game the road team was {game_margin} with {player1} on and {player2} on.\n")
      print(f"Possessions: {game_possessions/2}")
      game_count = game_count + 1
    else:
      if game_count == 1 or game_count == 2 or game_count == 6 or game_count == 7:
          arena1_margin = arena1_margin + game_margin
      if game_count == 3 or game_count == 4 or game_count == 5:
          arena2_margin = arena2_margin + game_margin

      game_possessions = game_possessions + pbp_poss
      series_possessions = series_possessions + game_possessions
      if game_margin >= 0:
        print(f"\nThis game the road team was +{game_margin} with {player1} on and {player2} on.\n")
      else:
        print(f"\nThis game the road team was {game_margin} with {player1} on and {player2} on.\n")
      print(f"Possessions: {game_possessions/2}")
      game_count = game_count + 1
  if finals_series == 1:
    if arena1_margin >= 0:
      print(f"In Games 1, 2, 6 and 7 visiting team was +{arena1_margin} with {player1} on and {player2} on.\n")
    else:
      print(f"In Games 1, 2, 6 and 7 visiting team was {arena1_margin} with {player1} on and {player2} on.\n")
    if arena2_margin >= 0:
      print(f"In Games 3, 4 and 5 visiting team was +{arena2_margin} with {player1} on and {player2} on.\n")
    else:
      print(f"In Games 3, 4 and 5 visiting team was {arena2_margin} with {player1} on and {player2} on.\n")
    print(f"{series_possessions/2} possessions this series with {player1} on and {player2} on.")
    game_count = game_count + 1
  else:
    if arena1_margin >= 0:
      print(f"In Games 1, 2, 5 and 7 visiting team was +{arena1_margin} with {player1} on and {player2} on.\n")
    else:
      print(f"In Games 1, 2, 5 and 7 visiting team was {arena1_margin} with {player1} on and {player2} on.\n")
    if arena2_margin >= 0:
      print(f"In Games 3, 4 and 6 visiting team was +{arena2_margin} with {player1} on and {player2} on.\n")
    else:
      print(f"In Games 3, 4 and 6 visiting team was {arena2_margin} with {player1} on and {player2} on.\n")
    print(f"{series_possessions/2} possessions this series with {player1} on and {player2} on.")
    game_count = game_count + 1
  stint_list_df.to_csv(f'{player1}_On_{player2}_On_Series_Stints.csv', index=False)

In [ ]:
#@title WOWY One Guy On / One Guy Off
def scrape_nba_possession_97_00_wowy_data(url, player1, player_ref1, player2, player_ref2, finals_series):
  wd.get(url)   
  html = wd.page_source
  soup = BeautifulSoup(html, features="lxml")
  time.sleep(10)

  stint_list_df = pd.DataFrame(columns = ['Game', 'Visiting Team +/-', 'Possessions'])
  stint_list_df = stint_list_df.iloc[0:0]

  for first_div in soup.find_all('div', attrs={'id': 'content'}):
          second_div = first_div.find('div', attrs={'id': 'div_other_scores'})
          hrefs = second_div.findAll('td', attrs={'class': 'right gamelink'})
          hrefs = str(hrefs)
          ref_urls = re.findall(r'/\d+\w+\D\w+', hrefs)
          urls = []
          for ref_url in ref_urls:
            ref_url = "https://www.basketball-reference.com/boxscores/pbp" + ref_url
            urls.append(ref_url)

  
  series_possessions = 0
  arena1_margin = 0
  arena2_margin = 0
  game_count = 1
  for game_url in urls:
    wd.get(game_url)   
    html = wd.page_source
    soup = BeautifulSoup(html, features="lxml")

    for first_div in soup.find_all('div', attrs={'id': 'content'}):
            second_div = first_div.find('div', attrs={'id': 'all_pbp'})
            first_table = second_div.find('table', attrs={'id': 'pbp'})
            body = first_table.find('tbody')
    # grab rows
    rows = body.findAll('tr')[0:]

    # Default possessions
    game_possessions = 0
    pbp_poss = 0
    stint_poss = 0
    secret_stint_poss = 0
    player_secretly_in_game_extra_poss = 0

    player2_secretly_in_game_start_of_q = 0

    player1_out_wait = 0
    player2_out_wait = 0
    predate_q_player1_out_wait = 0
    predate_q_player2_out_wait = 0
    
    margin = 0
    game_margin = 0
    initial_margin = 0

    grab_backup_score = 0
    grab_next_half_score = 0

    possession_enders = ['Defensive rebound', 'makes 2-pt', 'makes 3-pt', 'Turnover', 'Shooting foul']
    fts_no_shooting_foul = ['free throw 1 of 2']

    player1_out = [f'enters the game for <a href=\"{player_ref1}\">{player1}']
    player1_in = [f'{player1}']
    player1_enter = [f'{player1}</a> enters the game']

    player2_out = [f'enters the game for <a href=\"{player_ref2}\">{player2}']
    player2_in = [f'{player2}']
    player2_enter = [f'{player2}</a> enters the game']

    quarter_start = ['Start of 2nd quarter', 'Start of 4th quarter']
    half_start = ['Start of 3rd quarter']


    for row in rows:
      row = str(row)

      # 1/2 2nd half is starting; presume both players are now on the floor.
      if any(iny in row for iny in half_start):
        print(f"Half started, {player1} is on and {player2} is on.")
        if player2_out_wait == 1 and player1_out_wait == 0:
          grab_next_half_score = 1
          player2_out_wait = 0
          continue
      # 2/2 2nd half is starting; presume both players are now on the floor.
      if grab_next_half_score == 1:
        score = re.search("\d+-\d+", str(row))
        score = score[0]
        print(score)

        score1 = score.split('-')[0]
        score2 = score.split('-')[1]
        score1 = int(score1)
        score2 = int(score2)
        margin = score1 - score2

        margin = margin - initial_margin
        
        grab_next_half_score = 0

        print(f"\n{margin} +/- stint.\n")
        stint_list_df.loc[len(stint_list_df)] = [game_count, margin, int(stint_poss/2)]
        stint_poss = 0
        game_margin = game_margin + margin

      # 1/2 a quarter is starting; grab score in case a player has entered/left game without us knowing
      if any(iny in row for iny in quarter_start):
        grab_backup_score = 1
        continue

      # 2/2 grab score from next row as 'Q starting row' displays no score
      if grab_backup_score == 1:
        backup_score = re.search("\d+-\d+", str(row))
        backup_score = backup_score[0]

        score1 = backup_score.split('-')[0]
        score2 = backup_score.split('-')[1]
        score1 = int(score1)
        score2 = int(score2)

        backup_margin = score1 - score2
        player_secretly_in_game_extra_poss = 0
        secret_stint_poss = 0
        grab_backup_score = 0
        predate_q_player1_out_wait = player1_out_wait 
        predate_q_player2_out_wait = player2_out_wait 

      # player 1 is in the game (not entering) and was believed to be out of the game
      if any(iny in row for iny in player1_in) and not any(outy in row for outy in player1_out) and player1_out_wait == 1 and not any(iny in row for iny in player1_enter):
        player1_out_wait = 0

      # player 2 is in the game (not entering) and was believed to be out of the game
      if any(iny in row for iny in player2_in) and not any(outy in row for outy in player2_out) and player2_out_wait == 1 and not any(iny in row for iny in player2_enter):
        # player 1 is believed to be in the game
        # use margin from the beginning of 2nd or 4th quarter.
        if player1_out_wait == 0:
          print(f"{player1} on and {player2} on. {backup_score}")

          margin = backup_margin - initial_margin
          game_margin = game_margin + margin
          print(f"\n{margin} +/- stint.\n")
          stint_list_df.loc[len(stint_list_df)] = [game_count, margin, int(stint_poss/2)]
          stint_poss = 0
          player2_out_wait = 0
          player2_secretly_in_game_start_of_q = 1

      # player 1 is entering the game and was believed to be in it already
      if any(iny in row for iny in player1_enter) and player1_out_wait == 0:
        # player 2 is believed to be out of the game
        # use margin from the beginning of 2nd or 4th quarter.
        if predate_q_player2_out_wait == 1 and player2_secretly_in_game_start_of_q == 0:
          print(f"{player1} on and {player2} on. {backup_score}")

          margin = backup_margin - initial_margin
          game_margin = game_margin + margin
          print(f"\n{margin} +/- stint.\n")
          stint_list_df.loc[len(stint_list_df)] = [game_count, margin, int(stint_poss/2)]
          stint_poss = 0
        player2_secretly_in_game_start_of_q = 1

      # player 2 is entering the game and was believed to be in it already
      if any(iny in row for iny in player2_enter) and player2_out_wait == 0:
        # player 1 is believed to be in the game
        if player1_out_wait == 0:
          score = re.search("\d+-\d+", str(row))
          score = score[0]
          print(f"{player1} on and {player2} off. {backup_score}")
          print(f"{player1} on and {player2} on. {score}")
          score1 = score.split('-')[0]
          score2 = score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          margin = score1 - score2
          margin = margin - backup_margin
          print(f"\n{margin} +/- stint.\n")
          stint_poss = stint_poss + secret_stint_poss 
          stint_list_df.loc[len(stint_list_df)] = [game_count, margin, int(stint_poss/2)]
          stint_poss = 0
          game_margin = game_margin + margin
          pbp_poss = pbp_poss + player_secretly_in_game_extra_poss
          player_secretly_in_game_extra_poss = 0
          secret_stint_poss = 0

        # player 1 is believed to be out of the game
        if predate_q_player1_out_wait == 1 and player1_out_wait == 1:
          backup_score = re.search("\d+-\d+", str(row))
          backup_score = backup_score[0]

          score1 = backup_score.split('-')[0]
          score2 = backup_score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          backup_margin = score1 - score2
          player_secretly_in_game_extra_poss = 0
          secret_stint_poss = 0

      # player 1 is exiting the game and was believed to already be out
      if any(outy in row for outy in player1_out) and player1_out_wait == 1:
        if predate_q_player2_out_wait == 1 and player2_out_wait == 1:
          print(f"{player1} on and {player2} off. {backup_score}")

          margin = initial_margin - backup_margin
          print(f"\nVisting team's +/- with with {player1} on and {player2} off was {backup_margin}.\n")
          player1_out_wait = 0 

      # player 2 is exiting the game and was believed to already be out
      if any(outy in row for outy in player2_out) and player2_out_wait == 1:
        if predate_q_player1_out_wait == 0 and player1_out_wait == 0:
          print(f"{player1} on and {player2} off. {backup_score}")

          margin = initial_margin - backup_margin
          print(f"\nVisting team's +/- with with {player1} on and {player2} off was {backup_margin}.\n")
          player1_out_wait = 0

      # player 2 is exiting the game
      if any(outy in row for outy in player2_out):
        player2_out_wait = 1
        # player 1 is in the game and is not checking out
        # therefore grab score as a stint with player 1 on and player 2 off is starting
        if player1_out_wait == 0 and not any(outy in row for outy in player1_out):
          score = re.search("\d+-\d+", str(row))
          score = score[0]
          print(f"{player1} on and {player2} off. {score}")

          score1 = score.split('-')[0]
          score2 = score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          margin = score1 - score2
          initial_margin = margin


      # player 1 is listed on this row and is not checking out
      if any(iny in row for iny in player1_in) and not any(outy in row for outy in player1_out):
        # player 1 is checking in and player 2 is either checking out or was already out
        # therefore grab score as a stint with player 1 on and player 2 off is starting
        if player1_out_wait == 1 and ((player2_out_wait == 1) or any(outy in row for outy in player2_out)):
          score = re.search("\d+-\d+", str(row))
          score = score[0]
          print(f"{player1} on and {player2} off. {score}")

          score1 = score.split('-')[0]
          score2 = score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          margin = score1 - score2
          initial_margin = margin
        player1_out_wait = 0

      # player 2 is listed on this row and is not checking out
      if any(iny in row for iny in player2_in) and not any(outy in row for outy in player2_out):
      # player 2 is checking in and player 1 was on the floor and is not checking out
      # therefore update margin as a stint with player 1 on and player 2 off is ending.
        if player2_out_wait == 1 and player1_out_wait == 0:
          score = re.search("\d+-\d+", str(row))
          score = score[0]
          print(f"{player1} on and {player2} on. {score}")
          score1 = score.split('-')[0]
          score2 = score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          margin = score1 - score2
          margin = margin - initial_margin
          print(f"\n{margin} +/- stint.\n")
          stint_list_df.loc[len(stint_list_df)] = [game_count, margin, int(stint_poss/2)]
          stint_poss = 0
          game_margin = game_margin + margin
        player2_out_wait = 0

      # player 1 is exiting game
      if any(outy in row for outy in player1_out):
        player1_out_wait = 1
        # player 2 was out of the game and is not checking in the game
        # therefore update margin as a stint with player 1 on and player 2 off is ending.
        if player2_out_wait == 1 and not any(iny in row for iny in player2_in):
          score = re.search("\d+-\d+", str(row))
          score = score[0]
          print(f"{player1} off and {player2} off. {score}")
          score1 = score.split('-')[0]
          score2 = score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          margin = score1 - score2
          margin = margin - initial_margin
          print(f"\n{margin} +/- stint.\n")
          stint_list_df.loc[len(stint_list_df)] = [game_count, margin, int(stint_poss/2)]
          stint_poss = 0
          game_margin = game_margin + margin


      if any(end_poss in row for end_poss in possession_enders) and player1_out_wait == 0 and player2_out_wait == 1:
        pbp_poss = pbp_poss + 1
        stint_poss = stint_poss + 1
      elif any(end_poss in row for end_poss in possession_enders) and player1_out_wait == 0:
        player_secretly_in_game_extra_poss = player_secretly_in_game_extra_poss + 1
        secret_stint_poss = secret_stint_poss + 1

    last_row = body.findAll('tr')[-1]
    score = re.search("\d+-\d+", str(last_row))
    score = score[0]
    print(f"Final Score: {score}")

    if player2_out_wait == 1 and player1_out_wait == 0:
      score1 = score.split('-')[0]
      score2 = score.split('-')[1]
      score1 = int(score1)
      score2 = int(score2)

      margin = score1 - score2
      margin = margin - initial_margin
      print(f"\n{margin} +/- stint.\n")
      stint_list_df.loc[len(stint_list_df)] = [game_count, margin, int(stint_poss/2)]
      stint_poss = 0
      game_margin = game_margin + margin
    
    if finals_series == 0:

      if game_count == 1 or game_count == 2 or game_count == 5 or game_count == 7:
        arena1_margin = arena1_margin + game_margin
      if game_count == 3 or game_count == 4 or game_count == 6:
        arena2_margin = arena2_margin + game_margin

      game_possessions = game_possessions + pbp_poss
      series_possessions = series_possessions + game_possessions
      if game_margin >= 0:
        print(f"\nThis game the road team was +{game_margin} with {player1} on and {player2} off.\n")
      else:
        print(f"\nThis game the road team was {game_margin} with {player1} on and {player2} off.\n")
      print(f"Possessions: {game_possessions/2}")
      game_count = game_count + 1
    else:
      if game_count == 1 or game_count == 2 or game_count == 6 or game_count == 7:
          arena1_margin = arena1_margin + game_margin
      if game_count == 3 or game_count == 4 or game_count == 5:
          arena2_margin = arena2_margin + game_margin

      game_possessions = game_possessions + pbp_poss
      series_possessions = series_possessions + game_possessions
      if game_margin >= 0:
        print(f"\nThis game the road team was +{game_margin} with {player1} on and {player2} off.\n")
      else:
        print(f"\nThis game the road team was {game_margin} with {player1} on and {player2} off.\n")
      print(f"Possessions: {game_possessions/2}")
      game_count = game_count + 1
  if finals_series == 1:
    if arena1_margin >= 0:
      print(f"In Games 1, 2, 6 and 7 visiting team was +{arena1_margin} with {player1} on and {player2} off.\n")
    else:
      print(f"In Games 1, 2, 6 and 7 visiting team was {arena1_margin} with {player1} on and {player2} off.\n")
    if arena2_margin >= 0:
      print(f"In Games 3, 4 and 5 visiting team was +{arena2_margin} with {player1} on and {player2} off.\n")
    else:
      print(f"In Games 3, 4 and 5 visiting team was {arena2_margin} with {player1} on and {player2} off.\n")
    print(f"{series_possessions/2} possessions this series with {player1} on and {player2} off.")
    game_count = game_count + 1
  else:
    if arena1_margin >= 0:
      print(f"In Games 1, 2, 5 and 7 visiting team was +{arena1_margin} with {player1} on and {player2} off.\n")
    else:
      print(f"In Games 1, 2, 5 and 7 visiting team was {arena1_margin} with {player1} on and {player2} off.\n")
    if arena2_margin >= 0:
      print(f"In Games 3, 4 and 6 visiting team was +{arena2_margin} with {player1} on and {player2} off.\n")
    else:
      print(f"In Games 3, 4 and 6 visiting team was {arena2_margin} with {player1} on and {player2} off.\n")
    print(f"{series_possessions/2} possessions this series with {player1} on and {player2} off.")
    game_count = game_count + 1
  stint_list_df.to_csv(f'{player1}_On_{player2}_Off_Series_Stints.csv', index=False)

In [ ]:
scrape_nba_possession_97_00_wowy_data("https://www.basketball-reference.com/playoffs/1998-nba-western-conference-first-round-spurs-vs-suns.html", 'T. Duncan', '/players/d/duncati01.html', 'D. Robinson', '/players/r/robinda01.html', 0)

In [ ]:
#@title Fast One/One Test
  # one/one
  def_possessions = 0
  total_poss = 0
  player_secretly_in_game_extra_poss = 0

  player2_secretly_in_game_start_of_q = 0

  player1_out_wait = 0
  player2_out_wait = 0
  predate_q_player1_out_wait = 0
  predate_q_player2_out_wait = 0
  
  margin = 0
  game_margin = 0
  total_margin = 0
  initial_margin = 0

  grab_backup_score = 0
  grab_next_half_score = 0

  possession_enders = ['Defensive rebound', 'makes 2-pt', 'makes 3-pt', 'Turnover', 'Shooting foul']
  fts_no_shooting_foul = ['free throw 1 of 2']

  player1_out = [f'enters the game for <a href=\"{player_ref1}\">{player1}']
  player1_in = [f'{player1}']
  player1_enter = [f'{player1}</a> enters the game']

  player2_out = [f'enters the game for <a href=\"{player_ref2}\">{player2}']
  player2_in = [f'{player2}']
  player2_enter = [f'{player2}</a> enters the game']

  quarter_start = ['Start of 2nd quarter', 'Start of 4th quarter']
  half_start = ['Start of 3rd quarter']


  for row in rows:
    row = str(row)

    # 1/2 2nd half is starting; presume both players are now on the floor.
    if any(iny in row for iny in half_start):
      print(f"Half started, {player1} is on and {player2} is on")
      if player2_out_wait == 1 and player1_out_wait == 0:
        grab_next_half_score = 1
        player2_out_wait = 0
        continue
    # 2/2 2nd half is starting; presume both players are now on the floor.
    if grab_next_half_score == 1:
      score = re.search("\d+-\d+", str(row))
      score = score[0]
      print(score)

      score1 = score.split('-')[0]
      score2 = score.split('-')[1]
      score1 = int(score1)
      score2 = int(score2)
      margin = score1 - score2

      margin = margin - initial_margin
      
      grab_next_half_score = 0

      print(f"\nVisting team's +/- with {player1} on and {player2} off was {margin}\n")
      game_margin = game_margin + margin

    # 1/2 a quarter is starting; grab score in case a player has entered/left game without us knowing
    if any(iny in row for iny in quarter_start):
      grab_backup_score = 1
      continue

    # 2/2 grab score from next row as 'Q starting row' displays no score
    if grab_backup_score == 1:
      backup_score = re.search("\d+-\d+", str(row))
      backup_score = backup_score [0]

      score1 = backup_score.split('-')[0]
      score2 = backup_score.split('-')[1]
      score1 = int(score1)
      score2 = int(score2)

      backup_margin = score1 - score2
      player_secretly_in_game_extra_poss = 0
      grab_backup_score = 0
      predate_q_player1_out_wait = player1_out_wait 
      predate_q_player2_out_wait = player2_out_wait 

    # player 1 is in the game (not entering) and was believed to be out of the game
    if any(iny in row for iny in player1_in) and not any(outy in row for outy in player1_out) and player1_out_wait == 1 and not any(iny in row for iny in player1_enter):
      player1_out_wait = 0

    # player 2 is in the game (not entering) and was believed to be out of the game
    if any(iny in row for iny in player2_in) and not any(outy in row for outy in player2_out) and player2_out_wait == 1 and not any(iny in row for iny in player2_enter):
      # player 1 is believed to be in the game
      # use margin from the beginning of 2nd or 4th quarter.
      if player1_out_wait == 0:
        print(f"{player1} on and {player2} on")
        print(backup_score)

        margin = backup_margin - initial_margin
        game_margin = game_margin + margin
        print(f"\nVisting team's +/- with {player1} on and {player2} off was {margin}\n")
        player2_out_wait = 0
        player2_secretly_in_game_start_of_q = 1

    # player 1 is entering the game and was believed to be in it already
    if any(iny in row for iny in player1_enter) and player1_out_wait == 0:
      # player 2 is believed to be out of the game
      # use margin from the beginning of 2nd or 4th quarter.
      if predate_q_player2_out_wait == 1 and player2_secretly_in_game_start_of_q == 0:
        print(f"{player1} on and {player2} on")
        print(backup_score)

        margin = backup_margin - initial_margin
        game_margin = game_margin + margin
        print(f"\nVisting team's +/- with {player1} on and {player2} off was {margin}\n")
      player2_secretly_in_game_start_of_q = 1

    # player 2 is entering the game and was believed to be in it already
    if any(iny in row for iny in player2_enter) and player2_out_wait == 0:
      # player 1 is believed to be in the game
      if player1_out_wait == 0:
        score = re.search("\d+-\d+", str(row))
        score = score[0]
        print(f"{player1} on and {player2} off")
        print(backup_score)
        print(f"{player1} on and {player2} on")
        print(score)
        score1 = score.split('-')[0]
        score2 = score.split('-')[1]
        score1 = int(score1)
        score2 = int(score2)

        margin = score1 - score2
        margin = margin - backup_margin
        print(f"\nVisting team's +/- with {player1} on and {player2} off was {margin}\n")
        game_margin = game_margin + margin
        total_poss = total_poss + player_secretly_in_game_extra_poss
        player_secretly_in_game_extra_poss = 0

      # player 1 is believed to be out of the game
      if predate_q_player1_out_wait == 1 and player1_out_wait == 1:
        backup_score = re.search("\d+-\d+", str(row))
        backup_score = backup_score [0]

        score1 = backup_score.split('-')[0]
        score2 = backup_score.split('-')[1]
        score1 = int(score1)
        score2 = int(score2)

        backup_margin = score1 - score2
        player_secretly_in_game_extra_poss = 0

    # player 1 is exiting the game and was believed to already be out
    if any(outy in row for outy in player1_out) and player1_out_wait == 1:
      if predate_q_player2_out_wait == 1 and player2_out_wait == 1:
        print(f"{player1} on and {player2} off")
        print(backup_score)

        margin = initial_margin - backup_margin
        print(f"\nVisting team's +/- with with {player1} on and {player2} off was {backup_margin}\n")
        player1_out_wait = 0 

    # player 2 is exiting the game and was believed to already be out
    if any(outy in row for outy in player2_out) and player2_out_wait == 1:
      if predate_q_player1_out_wait == 0 and player1_out_wait == 0:
        print(f"{player1} on and {player2} off")
        print(backup_score)

        margin = initial_margin - backup_margin
        print(f"\nVisting team's +/- with with {player1} on and {player2} off was {backup_margin}\n")
        player1_out_wait = 0

    # player 2 is exiting the game
    if any(outy in row for outy in player2_out):
      player2_out_wait = 1
      # player 1 is in the game and is not checking out
      # therefore grab score as a stint with player 1 on and player 2 off is starting
      if player1_out_wait == 0 and not any(outy in row for outy in player1_out):
        score = re.search("\d+-\d+", str(row))
        score = score[0]
        print(f"{player1} on and {player2} off")
        print(score)

        score1 = score.split('-')[0]
        score2 = score.split('-')[1]
        score1 = int(score1)
        score2 = int(score2)

        margin = score1 - score2
        initial_margin = margin


    # player 1 is listed on this row and is not checking out
    if any(iny in row for iny in player1_in) and not any(outy in row for outy in player1_out):
      # player 1 is checking in and player 2 is either checking out or was already out
      # therefore grab score as a stint with player 1 on and player 2 off is starting
      if player1_out_wait == 1 and ((player2_out_wait == 1) or any(outy in row for outy in player2_out)):
        score = re.search("\d+-\d+", str(row))
        score = score[0]
        print(f"{player1} on and {player2} off")
        print(score)

        score1 = score.split('-')[0]
        score2 = score.split('-')[1]
        score1 = int(score1)
        score2 = int(score2)

        margin = score1 - score2
        initial_margin = margin
      player1_out_wait = 0

    # player 2 is listed on this row and is not checking out
    if any(iny in row for iny in player2_in) and not any(outy in row for outy in player2_out):
    # player 2 is checking in and player 1 was on the floor and is not checking out
    # therefore update margin as a stint with player 1 on and player 2 off is ending.
      if player2_out_wait == 1 and player1_out_wait == 0:
        score = re.search("\d+-\d+", str(row))
        score = score[0]
        print(f"{player1} on and {player2} on")
        print(score)
        score1 = score.split('-')[0]
        score2 = score.split('-')[1]
        score1 = int(score1)
        score2 = int(score2)

        margin = score1 - score2
        margin = margin - initial_margin
        print(f"\nVisting team's +/- with {player1} on and {player2} off was {margin}\n")
        game_margin = game_margin + margin
      player2_out_wait = 0

    # player 1 is exiting game
    if any(outy in row for outy in player1_out):
      player1_out_wait = 1
      # player 2 was out of the game and is not checking in the game
      # therefore update margin as a stint with player 1 on and player 2 off is ending.
      if player2_out_wait == 1 and not any(iny in row for iny in player2_in):
        score = re.search("\d+-\d+", str(row))
        score = score[0]
        print(f"{player1} off and {player2} off")
        print(score)
        score1 = score.split('-')[0]
        score2 = score.split('-')[1]
        score1 = int(score1)
        score2 = int(score2)

        margin = score1 - score2
        margin = margin - initial_margin
        print(f"\nVisting team's +/- with {player1} on and {player2} off was {margin}\n")
        game_margin = game_margin + margin


    if any(end_poss in row for end_poss in possession_enders) and player1_out_wait == 0 and player2_out_wait == 1:
      total_poss = total_poss + 1
    elif any(end_poss in row for end_poss in possession_enders) and player1_out_wait == 0:
      player_secretly_in_game_extra_poss = player_secretly_in_game_extra_poss + 1

  last_row = body.findAll('tr')[-1]
  score = re.search("\d+-\d+", str(last_row))
  score = score[0]
  print("game over")
  print(score)

  if player2_out_wait == 1 and player1_out_wait == 0:
    score1 = score.split('-')[0]
    score2 = score.split('-')[1]
    score1 = int(score1)
    score2 = int(score2)

    margin = score1 - score2
    margin = margin - initial_margin
    print(f"\nVisting team's +/- with {player1} on and {player2} off was {margin}\n")
    game_margin = game_margin + margin

  def_possessions = def_possessions + total_poss
  if game_margin >= 0:
    print(f"\nThis game the road team was +{game_margin} with {player1} on and {player2} off\n")
  else:
    print(f"\nThis game the road team was {game_margin} with {player1} on and {player2} off\n")
  total_margin = total_margin + game_margin
  print(f"Possessions: {def_possessions/2}")
  if total_margin >= 0:
    print(f"Series +/- for visiting team with {player1} on and {player2} off: +{total_margin}")
  else:
    print(f"Series +/- for visiting team with {player1} on and {player2} off: {total_margin}")

In [ ]:
#@title Fast Two/Two Test
    # two on
    series_possessions = 0
    arena1_margin = 0
    arena2_margin = 0
    game_count = 1

    stint_list_df = pd.DataFrame(columns = ['Game', 'Visiting Team +/-', 'Possessions'])
    stint_list_df = stint_list_df.iloc[0:0]

    #here#

    # Default possessions
    game_possessions = 0
    pbp_poss = 0
    stint_poss = 0
    secret_stint_poss = 0
    player_secretly_in_game_extra_poss = 0

    player1_out_wait = 0
    player2_out_wait = 0
    
    margin = 0
    game_margin = 0
    initial_margin = 0

    grab_backup_score = 0
    grab_next_half_score = 0

    p1_quarter_check = 0
    p2_quarter_check = 0


    quarter_conundrum = 0
    use_q = 0

    possession_enders = ['Defensive rebound', 'makes 2-pt', 'makes 3-pt', 'Turnover', 'Shooting foul']
    fts_no_shooting_foul = ['free throw 1 of 2']

    player1_out = [f'enters the game for <a href=\"{player_ref1}\">{player1}']
    player1_in = [f'{player1}']
    player1_enter = [f'{player1}</a> enters the game']

    player2_out = [f'enters the game for <a href=\"{player_ref2}\">{player2}']
    player2_in = [f'{player2}']
    player2_enter = [f'{player2}</a> enters the game']

    quarter_start = ['Start of 2nd quarter', 'Start of 4th quarter']
    half_start = ['Start of 3rd quarter']


    for row in rows:
      row = str(row)

      # 1/2 2nd half is starting; presume both players are now on the floor.
      if any(iny in row for iny in half_start):
        if player2_out_wait == 1 or player1_out_wait == 1:
          player1_out_wait = 0
          player2_out_wait = 0
          grab_next_half_score = 1
          continue
      # 2/2 2nd half is starting; presume both players are now on the floor.
      if grab_next_half_score == 1:
        score = re.search("\d+-\d+", str(row))
        score = score[0]
        print(f"Half started, {player1} is on and {player2} is on. {score}")

        score1 = score.split('-')[0]
        score2 = score.split('-')[1]
        score1 = int(score1)
        score2 = int(score2)

        initial_margin = score1 - score2
        player_secretly_in_game_extra_poss = 0
        secret_stint_poss = 0
        grab_backup_score = 0

        grab_next_half_score = 0

      # 1/2 a quarter is starting; grab score in case a player has entered/left game without us knowing
      if any(iny in row for iny in quarter_start):
        grab_backup_score = 1
        continue

      # 2/2 grab score from next row as 'Q starting row' displays no score
      if grab_backup_score == 1:
        backup_score = re.search("\d+-\d+", str(row))
        backup_score = backup_score[0]

        score1 = backup_score.split('-')[0]
        score2 = backup_score.split('-')[1]
        score1 = int(score1)
        score2 = int(score2)

        backup_margin = score1 - score2
        initial_margin = backup_margin
        q_margin = backup_margin
        q_score = backup_score
        player_secretly_in_game_extra_poss = 0
        secret_stint_poss = 0
        grab_backup_score = 0
        player1_out_wait = 1
        player2_out_wait = 1

        quarter_conundrum = 1

      # player 1 is in the game (not entering) and was believed to be out of the game
      if any(iny in row for iny in player1_in) and not any(outy in row for outy in player1_out) and player1_out_wait == 1 and not any(iny in row for iny in player1_enter):
          player1_out_wait = 0
          p1_quarter_check = 1
          if quarter_conundrum == 1 and p2_quarter_check == 1 and p2_quarter_check == 1:
            p1_quarter_check = 0
            p2_quarter_check = 0
            quarter_conundrum = 0

            # both player 1 and player 2 were in at the start of the Q
            # need to use score from beginning of quarter once either player finally
            # exits the game
            use_q = 1
          elif player2_out_wait == 0:
            score = re.search("\d+-\d+", str(row))
            score = score[0]
            print(f"{player1} on and {player2} on. {score}")
            print("problem4")

            score1 = score.split('-')[0]
            score2 = score.split('-')[1]
            score1 = int(score1)
            score2 = int(score2)

            margin = score1 - score2
            initial_margin = margin
            # player 1 is in game after start of a quarter; unsure about player 2's condition
          elif quarter_conundrum == 1 and p2_quarter_check == 0:
            backup_score = re.search("\d+-\d+", str(row))
            backup_score = backup_score[0]

            score1 = backup_score.split('-')[0]
            score2 = backup_score.split('-')[1]
            score1 = int(score1)
            score2 = int(score2)

            backup_margin = score1 - score2

      # player 2 is in the game (not entering) and was believed to be out of the game
      if any(iny in row for iny in player2_in) and not any(outy in row for outy in player2_out) and player2_out_wait == 1 and not any(iny in row for iny in player2_enter):
          player2_out_wait = 0
          p2_quarter_check = 1
          if quarter_conundrum == 1 and p1_quarter_check == 1 and p1_quarter_check == 1:
            p1_quarter_check = 0
            p2_quarter_check = 0
            quarter_conundrum = 0

            # both player 1 and player 2 were in at the start of the Q
            # need to use score from beginning of quarter once either player finally
            # exits the game
            use_q = 1
          elif player1_out_wait == 0:
            score = re.search("\d+-\d+", str(row))
            score = score[0]
            print(f"{player1} on and {player2} on. {score}")
            print("problem3")

            score1 = score.split('-')[0]
            score2 = score.split('-')[1]
            score1 = int(score1)
            score2 = int(score2)

            margin = score1 - score2
            initial_margin = margin
            # player 1 is in game after start of a quarter; unsure about player 2's condition
          elif quarter_conundrum == 1 and p1_quarter_check == 0:
            backup_score = re.search("\d+-\d+", str(row))
            backup_score = backup_score[0]

            score1 = backup_score.split('-')[0]
            score2 = backup_score.split('-')[1]
            score1 = int(score1)
            score2 = int(score2)

            backup_margin = score1 - score2

      # player 1 is exiting the game and was believed to already be out
      if any(outy in row for outy in player1_out) and player1_out_wait == 1:
        if player2_out_wait == 0:
          score = re.search("\d+-\d+", str(row))
          score = score[0]
          print(f"{player1} on and {player2} off. {score}")
          score1 = score.split('-')[0]
          score2 = score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          margin = score1 - score2
          margin = margin - initial_margin
          print(f"\n{margin} +/- stint.\n")
          stint_list_df.loc[len(stint_list_df)] = [game_count, margin, int(stint_poss/2)]
          stint_poss = 0
          game_margin = game_margin + margin

      # player 2 is exiting the game and was believed to already be out
      if any(outy in row for outy in player2_out) and player2_out_wait == 1:
        if player1_out_wait == 0:
          score = re.search("\d+-\d+", str(row))
          score = score[0]
          print(f"{player1} on and {player2} off. {score}")
          score1 = score.split('-')[0]
          score2 = score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          margin = score1 - score2
          margin = margin - initial_margin
          print(f"\n{margin} +/- stint.\n")
          stint_list_df.loc[len(stint_list_df)] = [game_count, margin, int(stint_poss/2)]
          stint_poss = 0
          game_margin = game_margin + margin

      # player 2 is exiting the game
      if any(outy in row for outy in player2_out):
        player2_out_wait = 1
        # need to use score from the beginning of the quarter as we didn't realize both players were in at the start
        if use_q == 1:
          use_q = 0
          print(f"{player1} on and {player2} off. {q_score}")
          print(f"\n{q_margin} +/- stint.\n")
          stint_list_df.loc[len(stint_list_df)] = [game_count, q_margin, int(stint_poss/2)]
          stint_poss = 0
          game_margin = game_margin + q_margin

        # player 1 is in the game and is not checking out
        elif player1_out_wait == 0 and not any(outy in row for outy in player1_out):
          score = re.search("\d+-\d+", str(row))
          score = score[0]
          print(f"{player1} on and {player2} off. {score}")
          score1 = score.split('-')[0]
          score2 = score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          margin = score1 - score2
          margin = margin - initial_margin
          print(f"\n{margin} +/- stint.\n")
          stint_list_df.loc[len(stint_list_df)] = [game_count, margin, int(stint_poss/2)]
          stint_poss = 0
          game_margin = game_margin + margin


      # player 1 is listed on this row and is not checking out
      if any(iny in row for iny in player1_in) and not any(outy in row for outy in player1_out):
        # player 1 is checking in and player 2 is also on
        # therefore grab score as a stint with player 1 on and player 2 off is starting
        if player1_out_wait == 1 and player2_out_wait == 0 and not any(iny in row for iny in player2_enter):
          score = re.search("\d+-\d+", str(row))
          score = score[0]
          print(f"{player1} on and {player2} on. {score}")
          print("problem2")

          score1 = score.split('-')[0]
          score2 = score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          margin = score1 - score2
          initial_margin = margin
        player1_out_wait = 0

      # player 2 is listed on this row and is not checking out
      if any(iny in row for iny in player2_in) and not any(outy in row for outy in player2_out):
      # player 2 is checking in and player 1 was on the floor and is not checking out
      # therefore update margin as a stint with player 1 on and player 2 off is ending.
        if player2_out_wait == 1 and player1_out_wait == 0 and not any(iny in row for iny in player1_enter) and not any(outy in row for outy in player1_out):
          score = re.search("\d+-\d+", str(row))
          score = score[0]
          print(f"{player1} on and {player2} on. {score}")
          print("problem1")
          score1 = score.split('-')[0]
          score2 = score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          margin = score1 - score2
          initial_margin = margin
        player2_out_wait = 0

      # player 1 is exiting game
      if any(outy in row for outy in player1_out):
        player1_out_wait = 1
        # need to use score from the beginning of the quarter as we didn't realize both players were in at the start
        if use_q == 1:
          use_q = 0
          print(f"{player1} off and {player2} on. {q_score}")
          print(f"\n{q_margin} +/- stint.\n")
          stint_list_df.loc[len(stint_list_df)] = [game_count, q_margin, int(stint_poss/2)]
          stint_poss = 0
          game_margin = game_margin + q_margin
        # player 2 is in game
        elif player2_out_wait == 0 and not any(iny in row for iny in player2_enter):
          score = re.search("\d+-\d+", str(row))
          score = score[0]
          print(f"{player1} off and {player2} on. {score}")
          score1 = score.split('-')[0]
          score2 = score.split('-')[1]
          score1 = int(score1)
          score2 = int(score2)

          margin = score1 - score2
          margin = margin - initial_margin
          print(f"\n{margin} +/- stint.\n")
          stint_list_df.loc[len(stint_list_df)] = [game_count, margin, int(stint_poss/2)]
          stint_poss = 0
          game_margin = game_margin + margin


      if any(end_poss in row for end_poss in possession_enders) and player1_out_wait == 0 and player2_out_wait == 0:
        pbp_poss = pbp_poss + 1
        stint_poss = stint_poss + 1
      elif any(end_poss in row for end_poss in possession_enders) and (player1_out_wait == 0 or player2_out_wait == 0):
        player_secretly_in_game_extra_poss = player_secretly_in_game_extra_poss + 1
        secret_stint_poss = secret_stint_poss + 1

    last_row = body.findAll('tr')[-1]
    score = re.search("\d+-\d+", str(last_row))
    score = score[0]
    print(f"Final Score: {score}")

    if player2_out_wait == 0 and player1_out_wait == 0:
      score1 = score.split('-')[0]
      score2 = score.split('-')[1]
      score1 = int(score1)
      score2 = int(score2)

      margin = score1 - score2
      margin = margin - initial_margin
      print(f"\n{margin} +/- stint.\n")
      stint_list_df.loc[len(stint_list_df)] = [game_count, margin, int(stint_poss/2)]
      stint_poss = 0
      game_margin = game_margin + margin
    
    if finals_series == 0:

      if game_count == 1 or game_count == 2 or game_count == 5 or game_count == 7:
        arena1_margin = arena1_margin + game_margin
      if game_count == 3 or game_count == 4 or game_count == 6:
        arena2_margin = arena2_margin + game_margin

      game_possessions = game_possessions + pbp_poss
      series_possessions = series_possessions + game_possessions
      if game_margin >= 0:
        print(f"\nThis game the road team was +{game_margin} with {player1} on and {player2} on.\n")
      else:
        print(f"\nThis game the road team was {game_margin} with {player1} on and {player2} on.\n")
      print(f"Possessions: {game_possessions/2}")
      game_count = game_count + 1
    else:
      if game_count == 1 or game_count == 2 or game_count == 6 or game_count == 7:
          arena1_margin = arena1_margin + game_margin
      if game_count == 3 or game_count == 4 or game_count == 5:
          arena2_margin = arena2_margin + game_margin

      game_possessions = game_possessions + pbp_poss
      series_possessions = series_possessions + game_possessions
      if game_margin >= 0:
        print(f"\nThis game the road team was +{game_margin} with {player1} on and {player2} on.\n")
      else:
        print(f"\nThis game the road team was {game_margin} with {player1} on and {player2} on.\n")
      print(f"Possessions: {game_possessions/2}")
      game_count = game_count + 1
    #indent here
    if finals_series == 1:
      if arena1_margin >= 0:
        print(f"In Games 1, 2, 6 and 7 visiting team was +{arena1_margin} with {player1} on and {player2} on.\n")
      else:
        print(f"In Games 1, 2, 6 and 7 visiting team was {arena1_margin} with {player1} on and {player2} on.\n")
      if arena2_margin >= 0:
        print(f"In Games 3, 4 and 5 visiting team was +{arena2_margin} with {player1} on and {player2} on.\n")
      else:
        print(f"In Games 3, 4 and 5 visiting team was {arena2_margin} with {player1} on and {player2} on.\n")
      print(f"{series_possessions/2} possessions this series with {player1} on and {player2} on.")
      game_count = game_count + 1
    else:
      if arena1_margin >= 0:
        print(f"In Games 1, 2, 5 and 7 visiting team was +{arena1_margin} with {player1} on and {player2} on.\n")
      else:
        print(f"In Games 1, 2, 5 and 7 visiting team was {arena1_margin} with {player1} on and {player2} on.\n")
      if arena2_margin >= 0:
        print(f"In Games 3, 4 and 6 visiting team was +{arena2_margin} with {player1} on and {player2} on.\n")
      else:
        print(f"In Games 3, 4 and 6 visiting team was {arena2_margin} with {player1} on and {player2} on.\n")
      print(f"{series_possessions/2} possessions this series with {player1} on and {player2} on.")
      game_count = game_count + 1
    stint_list_df.to_csv(f'{player1}_On_{player2}_On_Series_Stints.csv', index=False)

In [ ]:
player2 = 'T. Duncan'
player_ref2 =  '/players/d/duncati01.html'
player1 = 'D. Robinson'
player_ref1 = '/players/r/robinda01.html'
url = "https://www.basketball-reference.com/playoffs/1999-nba-finals-knicks-vs-spurs.html"

In [ ]:
wd.get(url)   
html = wd.page_source
soup = BeautifulSoup(html, features="lxml")
time.sleep(10)

for first_div in soup.find_all('div', attrs={'id': 'content'}):
        second_div = first_div.find('div', attrs={'id': 'div_other_scores'})
        hrefs = second_div.findAll('td', attrs={'class': 'right gamelink'})
        hrefs = str(hrefs)
        ref_urls = re.findall(r'/\d+\w+\D\w+', hrefs)
        urls = []
        for ref_url in ref_urls:
          ref_url = "https://www.basketball-reference.com/boxscores/pbp" + ref_url
          urls.append(ref_url)
game_url = urls[0]
wd.get(game_url)   
html = wd.page_source
soup = BeautifulSoup(html, features="lxml")

for first_div in soup.find_all('div', attrs={'id': 'content'}):
        second_div = first_div.find('div', attrs={'id': 'all_pbp'})
        first_table = second_div.find('table', attrs={'id': 'pbp'})
        body = first_table.find('tbody')
# grab rows
rows = body.findAll('tr')[0:]